# Analysis of Financial Documents to Forecast Stock Prices

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 17.5 MB/s eta 0:00:00


In [ ]:
!pip install text_hammer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.1 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
stock_predictions = pd.read_csv('NEWS_YAHOO_stock_prediction.csv')
stock_predictions.head()

,Unnamed: 0,ticker,Date,category,title,content,Open,High,Low,Close,Adj Close,Volume,label
0,0,AAPL,2020-01-27,opinion,Apple Set To Beat Q1 Earnings Estimates Tech ...,Technology giant Apple NASDAQ AAPL is set ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
1,1,AAPL,2020-01-27,opinion,Tech Daily Intel Results Netflix Surge Appl...,The top stories in this digest are Intel s N...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
2,2,AAPL,2020-01-27,opinion,7 Monster Stock Market Predictions For The Wee...,S P 500 SPY \nThis week will be packed with e...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
3,3,AAPL,2020-01-27,opinion,Apple Earnings Preview 5G Launch Expanding S...,Reports Q1 2020 results on Tuesday Jan 28 ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0
4,4,AAPL,2020-01-27,opinion,Buy Surging Apple Microsoft Stock Before Qua...,On today s episode of Full Court Finance here ...,77.514999,77.942497,76.220001,77.237503,75.793358,161940000,0


Filter out the opinion articles to see if there is any change in test accuracy.

In [ ]:
#stock_predictions = stock_predictions[:5000]
stock_predictions = stock_predictions[stock_predictions["category"]=="news"]
stock_predictions.shape

(5488, 13)

In [ ]:
import text_hammer as th
import warnings
warnings.filterwarnings('ignore')

In [ ]:
content = stock_predictions['content'].copy()[:]
content = content.to_list()
content = list(map(lambda x: x.lower(), content))
content = list(map(lambda x: th.remove_emails(x), content))
content = list(map(lambda x: th.remove_html_tags(x),content))
content = list(map(lambda x: th.remove_special_chars(x), content))
content = list(map(lambda x: th.remove_accented_chars(x), content))

In [ ]:
content[:1]

['by peter nurse investing com european stock markets are set to open sharply lower monday following weakness in asia caused by a sharp escalation of chinese measures to curtail the spread of the deadly coronavirus at 02 15 et 0715 gmt the dax futures contract traded 137 points or 1 lower france s cac 40 futures were down 72 points or 1 2 while the ftse 100 futures contract in the u k dropped 63 points or 0 9 futures on the pan eurozone index the euro stoxx 50 fell 40 points or 1 1 markets in mainland china and hong kong are shut but japan s nikkei 225 closed 2 lower and brent crude futures continues to tumble hitting their lowest level since october as traders priced in the hit to chinese demand gold futures and government bonds rallied according to recent clinical information the virus s ability to spread seems to be getting somewhat stronger china s national health commission minister ma xiaowei said at a press briefing sunday adding that it is infectious in its incubation period i 

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from sklearn.model_selection import train_test_split
#from keras.utils.all_utils import to_categorical
from keras.utils import to_categorical
Y = stock_predictions['label']
X = content.copy()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)

In [ ]:
df_train = pd.DataFrame({"text": X_train, "label": Y_train})
df_test = pd.DataFrame({"text": X_test, "label": Y_test})

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)


In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/3841 [00:00<?, ? examples/s]

Map:   0%|          | 0/1647 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Step,Training Loss
10,0.697200
20,0.700500
30,0.693800
40,0.695900
50,0.691400
60,0.690900
70,0.690600
80,0.690900
90,0.689900
100,0.697700


TrainOutput(global_step=363, training_loss=0.6862818675921312, metrics={'train_runtime': 555.9727, 'train_samples_per_second': 20.726, 'train_steps_per_second': 0.653, 'total_flos': 1526421834713088.0, 'train_loss': 0.6862818675921312, 'epoch': 3.0})

In [ ]:
trainer.predict(test_dataset)

In [ ]:
predictions = trainer.predict(test_dataset)
pred_logits = predictions.predictions
true_labels = predictions.label_ids


In [ ]:
import numpy as np
pred_labels = np.argmax(pred_logits, axis=1)


In [ ]:
np.unique(pred_labels, return_counts=True)

(array([0, 1]), array([ 477, 1170]))

In [ ]:
from sklearn.metrics import classification_report
import numpy as np


In [ ]:
report = classification_report(true_labels, pred_labels, output_dict=True)


In [ ]:
print("Precision: {:.2f}".format(report['weighted avg']['precision']))
print("Recall: {:.2f}".format(report['weighted avg']['recall']))
print("F1-score: {:.2f}".format(report['weighted avg']['f1-score']))


Precision: 0.53
Recall: 0.53
F1-score: 0.51
